In [ ]:
# samples a single point uniformly from a disc with defined radius
def sample_uniform_disc(radius=1):
    # Sample angles uniformly from 0 to 2*pi
    theta = np.random.uniform(0, 2 * np.pi, 1)
    
    # Sample radii with correct distribution
    r = radius * np.sqrt(np.random.uniform(0, 1, 1))
    
    # Convert polar coordinates to Cartesian coordinates
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return np.column_stack((x, y))[0]

voter_params = {'radius': 1}
candidate_params = {'radius': 1}
distance = lambda point1, point2: np.linalg.norm(point1 - point2)

disc_generator = Spatial(voter_dist = sample_uniform_disc, voter_params = voter_params,
                    candidate_dist = sample_uniform_disc, candidate_params = candidate_params,
                    distance = distance)

# Voronoi Generation

In [1]:
import math
from itertools import combinations, permutations
#from scipy.spatial import ConvexHull
#from matplotlib.path import Path
from scipy.optimize import linprog
#from scipy.spatial import Delaunay

In [ ]:
lines = np.zeros((math.comb(m,2), 3))

for i, candidate_pair in enumerate(combinations(range(m), 2)):
    c1 = candidate_pair[0]
    c2 = candidate_pair[1]  
    slope = (candidate_positions[c2,1] - candidate_positions[c1,1]) / (candidate_positions[c2,0] - candidate_positions[c1,0])
    slope = -1/slope
    midpoint = np.mean(candidate_positions[candidate_pair,:], axis = 0)
    lines[i, 0] = slope
    lines[i, 1] = -1
    lines[i, 2] = midpoint[1] - slope*midpoint[0]
    
border = np.array([[0,1,0], [0,1,-1], [1,0,0], [1,0,-1]])
lines = np.vstack((lines,border))
    
vertices = []
for i, line_pair in enumerate(combinations(range(len(lines)), 2)):
    l1 = line_pair[0]
    l2 = line_pair[1]
    
    a1, b1, c1 = lines[l1,:]
    a2, b2, c2 = lines[l2,:]
    
    # Create the coefficient matrix and the constant vector
    A = np.array([[a1, b1], [a2, b2]])
    B = np.array([-c1, -c2])
    
    # Solve the system of equations
    try:
        intersection = np.linalg.solve(A, B)
        if intersection[0] >= 0 and intersection[0] <= 1:
            if intersection[1] >= 0 and intersection[1] <= 1:
                vertices.append(intersection)
    except np.linalg.LinAlgError:
        pass
    
vertices = np.array(vertices)

In [ ]:
hull1 = [8, 3, 0]
hull2 = [3, 12, 6, 0]
hull3 = [6, 10, 7, 0]
hull4 = [7, 2, 0]
hull5 = [2, 9, 5, 0]
hull6 = [5, 11, 8, 0]

hulls = [hull1,hull2,hull3,hull4,hull5,hull6]
hulls = [vertices[h,:] for h in hulls]
hulls = [ConvexHull(h) for h in hulls]

areas = [h.volume for h in hulls]

In [ ]:
def in_hull(points, x):
    n_points = len(points)
    n_dim = len(x)
    c = np.zeros(n_points)
    A = np.r_[points.T,np.ones((1,n_points))]
    b = np.r_[x, np.ones(1)]
    lp = linprog(c, A_eq=A, b_eq=b)
    return lp.success

In [ ]:
def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

In [ ]:
hull_assignment = np.random.choice(range(len(hulls)), n, p = areas)
voter_positions = []

for a in hull_assignment:
    found = False
    while not found:
        random_point = np.random.uniform(0,1, size = 2)
        #if hull_path.contains_point([0.5,1]):
        if in_hull(random_point, hulls[a].points):
            voter_positions.append(random_point)
            found = True

In [ ]:
fig,ax = plt.subplots(dpi = 200)
ax.scatter(candidate_positions[:,0], candidate_positions[:,1], s = 60, c = 'k')
ax.scatter(voter_positions[:,0], voter_positions[:,1], c = hull_assignment, s = 30, cmap = custom_colors)

x = np.linspace(0,1,10)

for l in range(len(lines) - 4):
    y = (-lines[l, 0]*x - lines[l,2])/lines[l,1]
    ax.plot(x,y, linestyle = '--', c = 'k')
    
ax.vlines(x = 0, ymin = 0, ymax = 1, color = 'k')
ax.vlines(x = 1, ymin = 0, ymax = 1, color = 'k')
ax.hlines(y = 0, xmin = 0, xmax = 1, color = 'k')
ax.hlines(y = 1, xmin = 0, xmax = 1, color = 'k')

ax.set_ylim(-0.2,1.2)
ax.set_xlim(-0.2,1.2)

### Rejection Sample

In [ ]:
import votekit
from votekit.cvr_loaders import load_scottish
from sklearn.cluster import KMeans
prof, seats, cand_list, cand_to_party, ward = load_scottish(
    '../../scot-elex/5_cands/aberdeen_2022_ward11.csv'
)

In [ ]:
scot_profile = np.load('../data/scot-elex/aberdeen_2022_ward11.npy')

In [ ]:
m = scot_profile.shape[0]
n = scot_profile.shape[1]
k = 3
d = 8
#candidate_positions = np.random.uniform(0,1, size = (m,d))
#candidate_positions = np.zeros((m,d))
#np.fill_diagonal(candidate_positions,1)
candidate_positions = np.random.normal(loc = -1, scale = 0.05, size = (m,d))
np.fill_diagonal(candidate_positions[:,3:],1)
candidate_positions[0, :3] = [-0.18, -0.30, 1.01]
candidate_positions[1, :3] = [1.52, 0.89, -1.37]
candidate_positions[2, :3] = [-0.47, 0.22, -0.65]
candidate_positions[3, :3] = [-0.63, -0.87, 0.53]
candidate_positions[4, :3] = [0.42, -0.41, -0.32]


voter_dist_fn = np.random.uniform
voter_dist_fn_params = {'low' : -2, 'high' : 2, 'size' : d}
ranked_generator = RankedSpatial(
    voter_dist_fn,
    voter_dist_fn_params
)